In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import re
from itertools import chain
from collections import Counter
import torch.nn as nn
import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import transformers
from transformers import AdamW
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset, random_split
from tqdm import tqdm
import time

In [2]:
# specify device
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
class DescriptionDataset(Dataset):
    
    """Description dataset without species names."""
    
    def __init__(self, root_dir):
        
        self.root_dir = root_dir
        self.samples = []
        self._init_dataset()
        
    def __len__(self):
        return len(self.samples)
        
    def __getitem__(self, idx):
        return self.samples[idx]
     
    def _init_dataset(self):
        
        # Load the pickle list
        datalist = glob.glob(self.root_dir + 'train*.pkl')
        # Init list
        data_values = []
        # Loop over the pickles
        for data in datalist:
            # Open the pickles
            datadict = pickle.load(open(data, 'rb'))
            # Undict and append
            data_values += (list(chain.from_iterable(datadict.values())))
        
        # Drop double values 
        data_values = list(set(data_values))
        
        nested_values = [[tuple([1, span]) if text[0] == 1 else tuple([0, span]) 
                          for span in self.random_text_splitter(text[1])] 
                         for text in data_values]
        
        self.samples += list(chain.from_iterable(nested_values))
        
    def random_text_splitter(self, text):
        
        # Dont split short sentences
        if len(text.split()) <= 10:
            return [text]

        # Split text
        words = text.split()
        # Get length
        sentlength = len(words)
        # Random int
        randomint = random.randint(10, sentlength)
        # Check sentences from text
        parts = sentlength // randomint

        # Create sentences
        sentences = [' '.join(words[randomint*i:randomint*(i+1)]) for i in range(0, parts)]
        last_part = ' '.join(words[randomint*parts:])
        if len(last_part.split()) <= 10:
            sentences[-1] = sentences[-1] + last_part
        else:
            sentences += [last_part]

        return sentences

In [4]:
try:
    # Colab
    from google.colab import drive
    root = '/content/gdrive/My Drive/'
    drive.mount('/content/gdrive')
    print('Mounted @Google')
except:
    # Local
    root = "../data/processed/"
    print('Mounted @Local')

# Load data
data = DescriptionDataset(root)

Mounted @Local


In [5]:
# Inspect the data
ones = Counter(ones[0] for ones in data if ones[0] == 1)
zeros = Counter(ones[0] for ones in data if ones[0] == 0)

print('{0} samples.'. format(len(data)))
print('{0} labels with 1 (true).'.format(ones[1]))
print('{0} labels with 0 (false).'.format(zeros[0]))

1656393 samples.
71004 labels with 1 (true).
1585389 labels with 0 (false).


In [9]:
# create DataFrame using data
df = pd.DataFrame(data[0:10000], columns =['label', 'text'])

# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['label'], 
                                                                    random_state=2021, 
                                                                    test_size=0.2, 
                                                                    stratify=df['label'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2021, 
                                                                test_size=0.8, 
                                                                stratify=temp_labels)

In [10]:
# Load the BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Bert mode
bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
#start = time.time()
# Tokenize train data
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 512,
    padding=True,
    truncation=True)

# Tokenize validation data
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 512,
    padding=True,
    truncation=True)

# tokenize test data
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 512,
    padding=True,
    truncation=True)
#end = time.time()
#print("Time consumed in working: ",end - start)

  0%|                                                     | 0/2 [00:57<?, ?it/s]


In [ ]:
# List to Tensors
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [ ]:
# Batch
batch_size = 1

# Warp tensors
train_data = TensorDataset(train_seq, train_mask, train_y)
# Random sample (skewed set)
train_sampler = RandomSampler(train_data)
# DataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)
# Random sample
val_sampler = SequentialSampler(val_data)
# DataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
# Freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT(nn.Module):
    def __init__(self, bert):
        
        super(BERT, self).__init__()
        
        # Distil Bert model
        self.bert = bert
        ## Additional layers
        # Dropout layer
        self.dropout = nn.Dropout(0.1)
        # Relu 
        self.relu =  nn.ReLU()
        # Linear I 
        self.fc1 = nn.Linear(768, 512)
        # Linear II (Out)
        self.fc2 = nn.Linear(512, 2)
        # Softmax
        self.softmax = nn.LogSoftmax(dim=1)

    # Forward pass
    def forward(self, sent_id, mask):

        # Pass data trough bert and extract 
        cls_hs = self.bert(sent_id, attention_mask=mask)
        hidden_state = cls_hs[0]
        pooler = hidden_state[:, 0]
        
        # Dense layer 1        
        x = self.fc1(pooler)
        # ReLU activation
        x = self.relu(x)
        # Drop out
        x = self.dropout(x)
        # Dense layer 2
        x = self.fc2(x)
        # Activation
        x = self.softmax(x)

        return x

In [ ]:
# Load the entire model
model = BERT(bert)

# Load trained model (colab)
try:
    try:
        model_save_name = 'saved_weights.pt'
        path = F"/content/gdrive/My Drive/{model_save_name}"
        model.load_state_dict(torch.load(path))
        print('Google Success')

    except:
        model_save_name = 'saved_weights.pt'
        path = "../models/" + model_save_name
        model.load_state_dict(torch.load(path, 
                                         map_location=torch.device('cpu')))
        print('Local Success')
except:
    print('No pretrained model found.')

# Push the model to GPU
model = model.to(device)

In [ ]:
# Testing, Deactivate dropout layer
model.eval()
# Push a dataset trough the mode
BatchTest = next(iter(train_dataloader))

# Push the batch to gpu
batch = [r.to(device) for r in BatchTest]
sent_id, mask, labels = batch

# Push the data trough the model
preds = model(sent_id, mask)

In [ ]:
# Check the prediction 
torch.exp(preds)

In [ ]:
# Load optimizer (Adam best for bert)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-5)
# Define loss function
cross_entropy = torch.nn.CrossEntropyLoss()

In [ ]:
def train():
  
    """
    Function to train classification Bert model.
    """
    
    model.train()
    total_loss = 0
    
    # Iterate over batches
    for batch in tqdm(train_dataloader):

        # Push the batch to gpu
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch
        # Clear gradients 
        model.zero_grad()        
        # Get predictions
        preds = model(sent_id, mask)
        # Compute loss
        loss = cross_entropy(preds, labels)
        # Update total loss
        total_loss = total_loss + loss.item()
        # Backward pass to calculate the gradients
        loss.backward()
        # Clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters
        optimizer.step()

    # Compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    return avg_loss


def evaluate():
    
    """
    Function to test classification Bert model.
    """
  
    # Deactivate dropout layers
    model.eval()
    total_loss = 0

    # Iterate over batches
    for batch in tqdm(val_dataloader):   
        # Push the batch to gpu
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch

        # Deactivate autograd
        with torch.no_grad():
            # Model predictions
            preds = model(sent_id, mask)
            # Compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)
            total_loss = total_loss + loss.item()

    # Compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    return avg_loss

In [ ]:
# Epochs
epochs = 1

# Init loss
best_valid_loss = float('inf')

# data lists
train_losses=[]
valid_losses=[]

# Loop over epochs
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    # Train model
    train_loss = train() 
    # Evaluate model
    valid_loss  = evaluate()
        
    # Append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.6f}')
    print(f'Validation Loss: {valid_loss:.6f}')

In [ ]:
try:
    # Save @Google
    model_save_name = 'saved_weights.pt'
    path = F"/content/gdrive/My Drive/{model_save_name}" 
    torch.save(model.state_dict(), path)
    print('Saved @Google Drive')
except:
    # Save locally
    model_save_name = 'saved_weights.pt'
    path = '/notebooks/model/" + model_save_name
    torch.save(model.state_dict(), path)
    prin('Saved @local drive')

In [ ]:
# Loop over values
for count, test in enumerate(test_seq):
    with torch.no_grad():
        # Create prediction (CAN BE DONE AT ONCE WITH LARGER GPU)
        preds_temp = model(test_seq[count:count+1].to(device), test_mask[count:count+1].to(device))
        preds_temp = preds_temp.detach().cpu().numpy()
        
        # Init predictions
        if count == 0:
            preds = preds_temp
        else:
            # Concat arrays
            preds = np.concatenate((preds, preds_temp), axis=0)

In [ ]:
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

In [ ]:
'''
# number of training epochs
epochs = 1

# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
    
    with tqdm(train_dataloader, unit="batch") as tepoch:
        
        for batch in tepoch:
            
            tepoch.set_description(f"Epoch {epoch}")
         
            # Set model to train
            model.train()
            # empty list to save model predictions
            total_preds=[]

            # push the batch to gpu
            batch = [r.to(device) for r in batch]
            sent_id, mask, labels = batch

            # clear previously calculated gradients 
            model.zero_grad()        
            # get model predictions for the current batch
            preds = model(sent_id, mask)

            # compute the loss between actual and predicted values
            loss = cross_entropy(preds, labels)           
            # backward pass to calculate the gradients
            loss.backward()
            # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # update parameters
            optimizer.step()

            # Set taqaddum params   
            tepoch.set_postfix(loss=loss.item(), accuracy=total_accuracy)
            
            
# function to train the model
def train():
  
    model.train()

    total_loss, total_accuracy = 0, 0

    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step, batch in enumerate(train_dataloader):
    
        # progress update after every 50 batches.
        if step % 4 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch

        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds


# function for evaluating the model
def evaluate():
  
    print("\nEvaluating:")

    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step, batch in enumerate(val_dataloader):
    
        # Progress update every 50 batches.
        if step % 4 == 0 and not step == 0:
      
            # Calculate elapsed time in minutes.
            #elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [ ]:
'''
sklearn version slightly faster
'''

'''
# Split
TotalCount = len(data)
TrainCount = int(0.8 * TotalCount)
ValidCount = int(0.1 * TotalCount)
TestCount = TotalCount - TrainCount - ValidCount
TrainDataset, ValidDataset, TestDataset = random_split(data, (TrainCount, ValidCount, TestCount), 
                                                       generator=torch.Generator().manual_seed(33))
train_label = [label[0] for label in TrainDataset]
train_text = [text[1] for text in TrainDataset]

valid_label = [label[0] for label in ValidDataset]
val_text = [text[1] for text in ValidDataset]

test_label = [label[0] for label in TestDataset]
test_text = [text[1] for text in TestDataset]
'''

In [ ]:
'''

class DescriptionDataset(Dataset):
    
    """Description dataset without species names."""
    
    def __init__(self, root_dir):
        
        self.root_dir = root_dir
        self.samples = []
        self._init_dataset()
        
    def __len__(self):
        return len(self.samples)
        
    def __getitem__(self, idx):       
        return self.samples[idx]
    
    def _init_dataset(self):
        
        # Load the pickle list
        datalist = glob.glob(self.root_dir + 'train*.pkl')
        # Loop over the pickles
        for data in datalist:
            # Open the pickles
            datadict = pickle.load(open(data, 'rb'))
            # Undict and append
            self.samples += (list(chain.from_iterable(datadict.values())))
        
        # Drop double values 
        self.samples = list(set(self.samples))
def random_text_sampler(text):
    
    """
    Randomly breaks a piece of text into x pieces.
    """
    
    # Dont split short sentences
    if len(text.split()) <= 10:
        return [text]
    
    # Split text
    words = text.split()
    # Get length
    sentlength = len(words)
    # Random int
    randomint = random.randint(10, sentlength)
    # Check sentences from text
    parts = sentlength // randomint
    
    # Create sentences
    sentences = [' '.join(words[randomint*i:randomint*(i+1)]) for i in range(0, parts)]
    sentences += [' '.join(words[randomint*parts:])]
    
    return sentences
    
def random_text_sampler(text):
    
    """
    Randomly breaks a piece of text into 1 to 5 pieces.
    """
    
    if len(text.split()) < 10:
        return text
    
    # Get the length of the sentence
    n = int(len(text) / random.randint(1, 5))
    # Break up the text into spans
    spans = [text[i:i+n] for i in range(0, len(text), n)]
    
    # Check if end span is long enough
    if len(spans[-1].split()) < 2:
        spans_new = spans[:-2]
        spans_new.append(spans[-2] + spans[-1])
        
        return spans_new
    else:
        return spans
'''
